Daniel Xia

jx643

## Problem 1
### part a

State Space S={$(a_1,b_1),....(a_j,b_j)$} where maze_grid[$a_i,b_i$] is either "SPACE" or "GOAL"

Terminating Space T={(a,b)} where maze_grid[a,b]="GOAL"

Action Space={moving up,moving down, moving left, moving right} which result to change of state accoridngly


State Transition Function is that with each action the probability of moving to the next state is 1.0

Reward Transition Function 1:

Each step will have zero reward, recieve 1.0 reward if moving to the terminating state. Discount factor is between 1 and 0.

Reward Transition Function 2:

Each step wil have a negative reward of -1, and arraiving at the terminating state will have a positive reward of 100. Discount factor is 1.

### part b

In [1]:
from dataclasses import dataclass
from typing import Tuple, Dict,Mapping, Iterator
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.markov_decision_process import FinitePolicy, StateActionMapping
from rl.markov_process import FiniteMarkovProcess, FiniteMarkovRewardProcess
from rl.distribution import Categorical, Constant
from scipy.stats import poisson
from rl.dynamic_programming import value_iteration_result,value_iteration,almost_equal_vfs
from rl.iterate import converged, iterate,converge
import math

##Define the stateclass
@dataclass(frozen=True)
class MazeState:
    x: int
    y: int

# 0 is up, 1 is down, 2 is left,3 is right
MazeStateMapping = StateActionMapping[MazeState, int]


class SimpleMazeMDP(FiniteMarkovDecisionProcess[MazeState, int]):

    def __init__(
        self,
        maze: Mapping[Tuple[int, int], str],
        reward_type:int
    ):
        self.maze: Mapping[Tuple[int, int], str] = maze
        self.reward_type=reward_type
        super().__init__(self.get_action_transition_reward_map())
    
    ##Check if an action is valid
    def checkAction(self,state:MazeState,action:int):
        currentx=state.x
        currenty=state.y
        #check up
        if(action==0):
            if((currentx,currenty-1) in self.maze.keys()):
                return self.maze[(currentx,currenty-1)]=="SPACE" or self.maze[(currentx,currenty-1)]=="GOAL"
            else:
                return False
        #check down
        if(action==1):
            if((currentx,currenty+1) in self.maze.keys()):
                return self.maze[(currentx,currenty+1)]=="SPACE" or self.maze[(currentx,currenty+1)]=="GOAL"
            else:
                return False
        #check left
        if(action==2):
            if((currentx-1,currenty) in self.maze.keys()):
                return self.maze[(currentx-1,currenty)]=="SPACE" or self.maze[(currentx-1,currenty)]=="GOAL"
            else:
                return False
        #check right
        if((currentx+1,currenty) in self.maze.keys()):
            return self.maze[(currentx+1,currenty)]=="SPACE" or self.maze[(currentx+1,currenty)]=="GOAL"
        else:
            return False
        
    ## take action and return the new maze state
    def takeAction(self,state:MazeState,action:int):
        if(action==0):
            return MazeState(state.x,state.y-1)
        
        if(action==1):
            return MazeState(state.x,state.y+1)
        
        if(action==2):
            return MazeState(state.x-1,state.y)
        return MazeState(state.x+1,state.y)
    
    ## Check if the state is at the terminal state
    def checkGoal(self,state:MazeState):
        return self.maze[(state.x,state.y)]=="GOAL"

    def get_action_transition_reward_map(self) -> MazeStateMapping:
        d: Dict[MazeState, Dict[int, Categorical[Tuple[MazeState,float]]]] = {}
        for pos in self.maze.keys():
            #iterate through all the non-Block state
            if(self.maze[pos]!="BLOCK"):
                state: MazeState=MazeState(pos[0],pos[1])
                # terminal state returns None for the mapping
                if(self.checkGoal(state)):
                    d[state]=None
                else:
                    d1:Dict[int, Categorical[Tuple[MazeState,float]]]={}
                    for action in range(4):
                        sr_probs_dict: Dict[Tuple[MazeState, float], float]={}
                        if(self.checkAction(state,action)):
                            newState=self.takeAction(state,action)
                            # if the reward_type is 0, implement the reward policy where only arriving at terminal
                            # state will give award and the gamma is non zero
                            if(self.reward_type==0):
                                if(self.checkGoal(newState)):
                                    sr_probs_dict[(newState,1.0)]=1.0
                                else:
                                    sr_probs_dict[(newState,0.0)]=1.0
                            # if rewardtype is 1, implement the reward policy with no discounting factor but with a
                            # negative reward for any step not into the terminal state and a big positive reward for
                            # arriving at terminal state
                            else:
                                if(self.checkGoal(newState)):
                                    sr_probs_dict[(newState,100.0)]=1.0
                                else:
                                    sr_probs_dict[(newState,-1.0)]=1.0                             
                        d1[action]=Categorical(sr_probs_dict)
                    d[state]=d1
        return d

SPACE = 'SPACE'
BLOCK = 'BLOCK'
GOAL = 'GOAL'
maze_grid = {(0, 0): SPACE, (0, 1): BLOCK, (0, 2): SPACE, (0, 3): SPACE, (0, 4): SPACE, 
             (0, 5): SPACE, (0, 6): SPACE, (0, 7): SPACE, (1, 0): SPACE, (1, 1): BLOCK,
             (1, 2): BLOCK, (1, 3): SPACE, (1, 4): BLOCK, (1, 5): BLOCK, (1, 6): BLOCK, 
             (1, 7): BLOCK, (2, 0): SPACE, (2, 1): BLOCK, (2, 2): SPACE, (2, 3): SPACE, 
             (2, 4): SPACE, (2, 5): SPACE, (2, 6): BLOCK, (2, 7): SPACE, (3, 0): SPACE, 
             (3, 1): SPACE, (3, 2): SPACE, (3, 3): BLOCK, (3, 4): BLOCK, (3, 5): SPACE, 
             (3, 6): BLOCK, (3, 7): SPACE, (4, 0): SPACE, (4, 1): BLOCK, (4, 2): SPACE, 
             (4, 3): BLOCK, (4, 4): SPACE, (4, 5): SPACE, (4, 6): SPACE, (4, 7): SPACE, 
             (5, 0): BLOCK, (5, 1): BLOCK, (5, 2): SPACE, (5, 3): BLOCK, (5, 4): SPACE, 
             (5, 5): BLOCK, (5, 6): SPACE, (5, 7): BLOCK, (6, 0): SPACE, (6, 1): BLOCK, 
             (6, 2): BLOCK, (6, 3): BLOCK, (6, 4): SPACE, (6, 5): BLOCK, (6, 6): SPACE, 
             (6, 7): SPACE, (7, 0): SPACE, (7, 1): SPACE, (7, 2): SPACE, (7, 3): SPACE, 
             (7, 4): SPACE, (7, 5): BLOCK, (7, 6): BLOCK, (7, 7): GOAL}
from pprint import pprint
user_gamma=0.9
si_mdp1: FiniteMarkovDecisionProcess[MazeState, int] =SimpleMazeMDP(maze=maze_grid,reward_type=0)
generator=value_iteration(si_mdp1,gamma=user_gamma)
count=0
for i in converge(generator,almost_equal_vfs):
    count+=1
print("number of iteration to convergence for the first reward type is",count)
opt_vf_vi, opt_policy_vi = value_iteration_result(si_mdp1, gamma=user_gamma)
print("number of steps to finish maze is",int(round(math.log(opt_vf_vi[MazeState(0,0)],user_gamma)+1)))

number of iteration to convergence for the first reward type is 17
number of steps to finish maze is 16


In [21]:
si_mdp2: FiniteMarkovDecisionProcess[MazeState, int] =SimpleMazeMDP(maze=maze_grid,reward_type=1)
generator2=value_iteration(si_mdp2,gamma=user_gamma)
count=0
for i in converge(generator2,almost_equal_vfs):
    count+=1
print("number of iteration to convergence for the second reward type is",count)
opt_vf_vi2, opt_policy_vi2 = value_iteration_result(si_mdp2, gamma=1.0)
print("number of steps to finish maze is",int(round(100-opt_vf_vi2[MazeState(0,0)]+1)))

number of iteration to convergence for the second reward type is 17
number of steps to finish maze is 16


### Part c

In [22]:
def get_path(start,end,policy_map):
    path=[start]
    while(path[-1]!=end):
        current=path[-1]
        action=list(policy_map[MazeState(current[0],current[1])])[0][0]
        if(action==0):
            path.append((current[0],current[1]-1))
        if(action==1):
            path.append((current[0],current[1]+1))
        if(action==2):
            path.append((current[0]-1,current[1]))
        if(action==3):
            path.append((current[0]+1,current[1]))
    return path

In [23]:
print(get_path((0,0),(7,7),opt_policy_vi.policy_map))
print(get_path((0,0),(7,7),opt_policy_vi2.policy_map))

[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (4, 6), (5, 6), (6, 6), (6, 7), (7, 7)]
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (2, 2), (2, 3), (2, 4), (2, 5), (3, 5), (4, 5), (4, 6), (5, 6), (6, 6), (6, 7), (7, 7)]


As you can see, the optimal policy path from (0,0) is printed out and they are the same for both. The number of iteration to convergence as printed out above are the same. Both have 17 iteration before convergence.

## Problem 2

According to equation 1.2 on page 71, we can have the formulation of the value function in terms of the $\gamma$, P and R.

We know that there is an equilibrium point where:
$$V=R+\gamma P*V$$
Now we want to estimate the function with $\hat{V}$ where $\hat{V}(s_t)=V(s_t)$ and we want to approximate if with a linear approximation with the feature matrix $\Phi$ where $\hat{V}=\Phi(S)W$ where W is a m*1 weight. For the $\hat{V}(S)$ to converge,
we can write the update rule for the covergence.  $V(S)\gets V(S)+\alpha(R+\gamma P\hat{V}(S)-\hat{V}(S))$ where $\alpha$ is some learning rate parameter. For it to converge to the true value function, we need the TD error to be zero where $R+\gamma P \hat{V}(S)-\hat{V}(S)=0$. So that $R+\gamma P\Phi(S)W-\Phi(S)W=0$ and this is the condition for the linear feature approximation of the value function to converge to the actual value function. $\Phi(S)W=(I-\gamma P)^{-1}R$. So we know that $(I-\gamma P)^{-1}R$ is in the column space of $\Phi$.

## Problem 3

### Part a)
States: S={1,....,W}. I model the current wages as the state.

Action: {(l,s),...} where l+s<=H pairs of hours spent on learning and the hour spent on searching as the action. 

Reward: $s_t*(H-l-s)$ reward is calculated as the wage times the number of hours to work.

State transition probability: we need to take this into different cases. 

1. if $s_t=W$,$P(s_{t+1}=W)=1.0$ 
    with probability of 1.0, the next state will be $s_{t+1}=W$.

    In this case, wage reaches maximum, cannot increase anymore.
    

2. $P(s_{t+1}=s_t+k)=poisson.pmf(k,\alpha*l)$ for $2\leq k<H-s_t$
    with probability of poisson.pmf(k,alpha*l), the next state will be $s_t+k$ with $1\leq k< H-s_t$.

    In this case, you got higher wage from current company, but the wage does not reach maximum.
    

3. $P(s_{t+1}=s_t)=poisson.pmf(0,\alpha * l) * \beta*s/H$
    with probability of poisson.pmf(0,alpha * l) * beta*s/H, the next state will be $s_{t+1}=s_t$.
    
    In this case, you didnt recieve raise and offer.

4. $P(s_{t+1}=W)=1.0-poisson.cdf(W-s_t,\alpha*l)$
    with probability of 1.0-poisson.cdf(W-s_t,alpha*l), the next state will be $s_t=W$.
    
    In this case, your raise reaches maximum.

5. $P(s_{t+1}=s_t+1)=poisson.pmf(0,\alpha*l)*(\beta*s/H)+poisson.pmf(k,\alpha*l)$
    with probability of poisson.pmf(0,alpha*l)*(beta*s/H), the next state will be $s_t=s_t+1$.

    In this case, you recieve wage of zero or one or recieve an offer from other company.
    
Discount factor is between 0 and 1.

In [2]:
##Define the stateclass
@dataclass(frozen=True)
class SalaryState:
    x: int

# 0 is up, 1 is down, 2 is left,3 is right
SalaryStateMapping = StateActionMapping[SalaryState,Tuple[int,int]]


class SalaryMDP(FiniteMarkovDecisionProcess[SalaryState, int]):

    def __init__(
        self,
        H:int,
        W:int,
        alpha:float,
        beta:float
    ):
        self.H = H
        self.W=W
        self.alpha=alpha
        self.beta=beta
        super().__init__(self.get_action_transition_reward_map())

    def get_action_transition_reward_map(self) -> SalaryStateMapping:
        d: Dict[SalaryState, Dict[Tuple[int,int], Categorical[Tuple[SalaryState,float]]]] = {}
        for salary in range (self.W+1):
            d1:Dict[Tuple[int,int], Categorical[Tuple[SalaryState,float]]]={}
            if salary==self.W:
                sr_probs_dict: Dict[Tuple[SalaryState, float], float]={}
                sr_probs_dict[(SalaryState(salary),salary*self.H)]=1.0
                d1[(0,0)]=Categorical(sr_probs_dict)
            else:
                for l in range(self.H+1):
                    for s in range(self.H+1-l):
                        sr_probs_dict: Dict[Tuple[SalaryState, float], float]={}
                        reward=salary*(self.H-l-s)
                        action=(l,s)
                        prob_offer=self.beta*s/self.H
                        offer_price=min(salary+1,self.W)
                        for bonus in range(2,self.W-salary):
                            prob_bonus=poisson.pmf(bonus,self.alpha*l)
                            sr_probs_dict[(SalaryState(salary+bonus),reward)]=prob_bonus
                        prob_reach_max=1.0-poisson.cdf(self.W-salary,self.alpha*l)
                        sr_probs_dict[(SalaryState(self.W),reward)]=prob_reach_max
                        prob_stay_zero=poisson.pmf(0,self.alpha*l)*(1-prob_offer)
                        sr_probs_dict[(SalaryState(salary),reward)]=prob_stay_zero
                        prob_accept_other_offer=poisson.pmf(0,self.alpha*l)*(prob_offer)+poisson.pmf(1,self.alpha*l)
                        sr_probs_dict[(SalaryState(salary+1),reward)]=prob_accept_other_offer
                        d1[action]=Categorical(sr_probs_dict)
            d[SalaryState(salary)]=d1             
        return d

In [3]:
mdp1: FiniteMarkovDecisionProcess[SalaryState, int] =SalaryMDP(H=10,W=30,alpha=0.08,beta=0.82)

In [4]:
opt_vf_vi,opt_policy_vi = value_iteration_result(mdp1, gamma=0.95)
opt_vf_vi

{SalaryState(x=0): 1183.7522923792844,
 SalaryState(x=1): 1259.6504926227421,
 SalaryState(x=2): 1340.415028776917,
 SalaryState(x=3): 1426.357913842387,
 SalaryState(x=4): 1517.8111660380023,
 SalaryState(x=5): 1615.1280914674717,
 SalaryState(x=6): 1718.684649031723,
 SalaryState(x=7): 1828.8809028830503,
 SalaryState(x=8): 1946.1425677166962,
 SalaryState(x=9): 2070.922650716453,
 SalaryState(x=10): 2203.703212047009,
 SalaryState(x=11): 2344.997430859798,
 SalaryState(x=12): 2495.3513213054002,
 SalaryState(x=13): 2655.325653274279,
 SalaryState(x=14): 2825.6334066075733,
 SalaryState(x=15): 3006.9962764014304,
 SalaryState(x=16): 3199.999895219283,
 SalaryState(x=17): 3399.9998886704875,
 SalaryState(x=18): 3599.999882121693,
 SalaryState(x=19): 3799.9998755728984,
 SalaryState(x=20): 3999.9998690241036,
 SalaryState(x=21): 4199.999862475308,
 SalaryState(x=22): 4399.999855926513,
 SalaryState(x=23): 4599.9998493777175,
 SalaryState(x=24): 4799.999842828925,
 SalaryState(x=25): 49

In [5]:
opt_policy_vi.policy_map

{SalaryState(x=0): {(10, 0): 1},
 SalaryState(x=1): {(10, 0): 1},
 SalaryState(x=2): {(10, 0): 1},
 SalaryState(x=3): {(10, 0): 1},
 SalaryState(x=4): {(10, 0): 1},
 SalaryState(x=5): {(10, 0): 1},
 SalaryState(x=6): {(10, 0): 1},
 SalaryState(x=7): {(10, 0): 1},
 SalaryState(x=8): {(10, 0): 1},
 SalaryState(x=9): {(10, 0): 1},
 SalaryState(x=10): {(10, 0): 1},
 SalaryState(x=11): {(10, 0): 1},
 SalaryState(x=12): {(10, 0): 1},
 SalaryState(x=13): {(10, 0): 1},
 SalaryState(x=14): {(0, 10): 1},
 SalaryState(x=15): {(0, 10): 1},
 SalaryState(x=16): {(0, 0): 1},
 SalaryState(x=17): {(0, 0): 1},
 SalaryState(x=18): {(0, 0): 1},
 SalaryState(x=19): {(0, 0): 1},
 SalaryState(x=20): {(0, 0): 1},
 SalaryState(x=21): {(0, 0): 1},
 SalaryState(x=22): {(0, 0): 1},
 SalaryState(x=23): {(0, 0): 1},
 SalaryState(x=24): {(0, 0): 1},
 SalaryState(x=25): {(0, 0): 1},
 SalaryState(x=26): {(0, 0): 1},
 SalaryState(x=27): {(0, 0): 1},
 SalaryState(x=28): {(0, 0): 1},
 SalaryState(x=29): {(0, 0): 1},
 Sal

From the optimal policy we can see that for wages below 14, we should spend all the time searching for job and for wages from 14 to 15, we need to speend all time searching for job,and from 16 onward it is better to spend all time working. It is intuitive since for a higher paid job, the amount of raise from getting a new job doesnot conpensate the amount of money lost by not working for that period. For lowe paid job, the amount of raise is more valuable compared to the lost of salary for not working during the period.